# Task 1: Comparative analysis of the performance of pre-trained linguistic parsers and Named Entity Recognition (NER) models on given English datasets using standard metrics.

For this task use the NER dataset (Link).

Implement and evaluate the POS tagger and NER models from the following NLP libraries -
SpaCy
NLTK
StanfordNER
on the given dataset without additional training.
Explain the text processing pipeline adopted by you.
Evaluate model predictions using accuracy, and confusion matrix (TAG-wise) metrics. Provide comparative analysis.
Analyze the comparative strengths of each model based on the above evaluation metrics.
Record your observations about errors committed by each model - do you observe any pattern? Discuss in detail along with results to justify your observations.
Note that: TAGs in the provided dataset can be different from the TAGs given by the model, therefore it is requested to map the TAGs of the dataset with the TAGs of the model accordingly. One example is provided here to create a mapping between dataset TAGs to SpaCy model tags: [Link] (key of the dict is dataset TAG and value of the dict is model’s TAG) (similarly with POS tags, you are free to make similar dict for ease of comparison)

In [ ]:
!pip install -U spacy
!pip install -U spacy-lookups-data
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 34.6 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.2
    Uninstalling spacy-3.7.2:
      Successfully uninstalled spacy-3.7.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 76.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
import pandas as pd
import ast
import pickle
import pandas as pd
pd.set_option('display.max_columns', 50)

In [ ]:
df= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NER_Dataset.csv')
#Mapping
dataset_to_spacy = {
    "B-art": "ART",
    "B-eve": "EVENT",
    "B-geo": "GPE",
    "B-gpe": "GPE",
    "B-nat": "NORP",
    "B-org": "ORG",
    "B-per": "PERSON",
    "B-tim": "TIME",
    "I-art": "ART",
    "I-eve": "EVENT",
    "I-geo": "GPE",
    "I-gpe": "GPE",
    "I-nat": "NORP",
    "I-org": "ORG",
    "I-per": "PERSON",
    "I-tim": "TIME",
    "O": ""
}

nltk_to_spacy = {
    "B-ART": "ART",
    "B-EVE": "EVENT",
    "B-GEO": "GPE",
    "B-GPE": "GPE",
    "B-NAT": "NORP",
    "B-ORG": "ORG",
    "B-PER": "PERSON",
    "B-TIM": "TIME",
    "I-ART": "ART",
    "I-EVE": "EVENT",
    "I-GEO": "GPE",
    "I-GPE": "GPE",
    "I-NAT": "NORP",
    "I-ORG": "ORG",
    "I-PER": "PERSON",
    "I-TIM": "TIME",
    "O": ""
}

In [ ]:
df.head()

Sentence_ID                                               Word  \
0      Sentence: 1  ['Thousands', 'of', 'demonstrators', 'have', '...   
1     Sentence: 10  ['Iranian', 'officials', 'say', 'they', 'expec...   
2    Sentence: 100  ['Helicopter', 'gunships', 'Saturday', 'pounde...   
3   Sentence: 1000  ['They', 'left', 'after', 'a', 'tense', 'hour-...   
4  Sentence: 10000  ['U.N.', 'relief', 'coordinator', 'Jan', 'Egel...   

                                                 POS  \
0  ['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...   
1  ['JJ', 'NNS', 'VBP', 'PRP', 'VBP', 'TO', 'VB',...   
2  ['NN', 'NNS', 'NNP', 'VBD', 'JJ', 'NNS', 'IN',...   
3  ['PRP', 'VBD', 'IN', 'DT', 'NN', 'JJ', 'NN', '...   
4  ['NNP', 'NN', 'NN', 'NNP', 'NNP', 'VBD', 'NNP'...   

                                                 Tag  
0  ['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '...  
1  ['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '...  
2  ['O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', '...  
3  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...  
4  ['B-geo', 'O', 'O', 'B-per', 'I-per', 'O', 'B-...

In [ ]:
df['Sentence'] = df['Word'].apply(lambda a:" " .join(ast.literal_eval(a)))

In [ ]:
df = df.head(100)
df.head()

Sentence_ID                                               Word  \
0      Sentence: 1  ['Thousands', 'of', 'demonstrators', 'have', '...   
1     Sentence: 10  ['Iranian', 'officials', 'say', 'they', 'expec...   
2    Sentence: 100  ['Helicopter', 'gunships', 'Saturday', 'pounde...   
3   Sentence: 1000  ['They', 'left', 'after', 'a', 'tense', 'hour-...   
4  Sentence: 10000  ['U.N.', 'relief', 'coordinator', 'Jan', 'Egel...   

                                                 POS  \
0  ['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...   
1  ['JJ', 'NNS', 'VBP', 'PRP', 'VBP', 'TO', 'VB',...   
2  ['NN', 'NNS', 'NNP', 'VBD', 'JJ', 'NNS', 'IN',...   
3  ['PRP', 'VBD', 'IN', 'DT', 'NN', 'JJ', 'NN', '...   
4  ['NNP', 'NN', 'NN', 'NNP', 'NNP', 'VBD', 'NNP'...   

                                                 Tag  \
0  ['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '...   
1  ['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '...   
2  ['O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', '...   
3  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...   
4  ['B-geo', 'O', 'O', 'B-per', 'I-per', 'O', 'B-...   

                                            Sentence  
0  Thousands of demonstrators have marched throug...  
1  Iranian officials say they expect to get acces...  
2  Helicopter gunships Saturday pounded militant ...  
3  They left after a tense hour-long standoff wit...  
4  U.N. relief coordinator Jan Egeland said Sunda...

In [ ]:
from nltk.parse import CoreNLPParser
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize
from nltk.tag import StanfordNERTagger
from nltk.internals import find_jars_within_path

In [ ]:
nlp_spacy = spacy.load("en_core_web_sm")

In [ ]:
# Defining the function to process text and apply SpaCy's NER model
def apply_ner(text):
    doc = nlp(text)
    entities = [(token.text,token.pos_,token.ent_type_) for token in doc]
    return entities

nlp = spacy.load("en_core_web_sm")

# Apply SpaCy's NER model to each sentence in the DataFrame
df['Spacy_pos_ner'] = df['Sentence'].apply(apply_ner)

df['Dataset_pos_ner'] = df.apply(lambda row: list(zip(ast.literal_eval(row['Word']), ast.literal_eval(row['POS']), ast.literal_eval(row['Tag']))), axis=1)
df['Dataset_pos_ner'] = df['Dataset_pos_ner'].apply(lambda x : [(t[0], t[1], dataset_to_spacy.get(t[2], t[2])) for t in x])
# Print the DataFrame with the NER entities
df.head()

Sentence_ID                                               Word  \
0      Sentence: 1  ['Thousands', 'of', 'demonstrators', 'have', '...   
1     Sentence: 10  ['Iranian', 'officials', 'say', 'they', 'expec...   
2    Sentence: 100  ['Helicopter', 'gunships', 'Saturday', 'pounde...   
3   Sentence: 1000  ['They', 'left', 'after', 'a', 'tense', 'hour-...   
4  Sentence: 10000  ['U.N.', 'relief', 'coordinator', 'Jan', 'Egel...   

                                                 POS  \
0  ['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...   
1  ['JJ', 'NNS', 'VBP', 'PRP', 'VBP', 'TO', 'VB',...   
2  ['NN', 'NNS', 'NNP', 'VBD', 'JJ', 'NNS', 'IN',...   
3  ['PRP', 'VBD', 'IN', 'DT', 'NN', 'JJ', 'NN', '...   
4  ['NNP', 'NN', 'NN', 'NNP', 'NNP', 'VBD', 'NNP'...   

                                                 Tag  \
0  ['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '...   
1  ['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '...   
2  ['O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', '...   
3  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...   
4  ['B-geo', 'O', 'O', 'B-per', 'I-per', 'O', 'B-...   

                                            Sentence  \
0  Thousands of demonstrators have marched throug...   
1  Iranian officials say they expect to get acces...   
2  Helicopter gunships Saturday pounded militant ...   
3  They left after a tense hour-long standoff wit...   
4  U.N. relief coordinator Jan Egeland said Sunda...   

                                       Spacy_pos_ner  \
0  [(Thousands, NOUN, CARDINAL), (of, ADP, ), (de...   
1  [(Iranian, ADJ, NORP), (officials, NOUN, ), (s...   
2  [(Helicopter, NOUN, ), (gunships, NOUN, ), (Sa...   
3  [(They, PRON, ), (left, VERB, ), (after, ADP, ...   
4  [(U.N., PROPN, ORG), (relief, NOUN, ), (coordi...   

                                     Dataset_pos_ner  
0  [(Thousands, NNS, ), (of, IN, ), (demonstrator...  
1  [(Iranian, JJ, GPE), (officials, NNS, ), (say,...  
2  [(Helicopter, NN, ), (gunships, NNS, ), (Satur...  
3  [(They, PRP, ), (left, VBD, ), (after, IN, ), ...  
4  [(U.N., NNP, GPE), (relief, NN, ), (coordinato...

In [ ]:
 import nltk
 nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
 nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [ ]:
import pandas as pd
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.chunk import conlltags2tree, tree2conlltags
# Defining the function to apply NLTK's NER
def nltk_ner(text):
    # Text Tokenization
    tokens = word_tokenize(text)
    # POS tagging
    pos_tags = pos_tag(tokens)
    # NER Operation
    tree = ne_chunk(pos_tags)
    # NLTK's tree format to IOB (Inside, Outside, Beginning) tags
    iob_tags = tree2conlltags(tree)
    # IOB tags to word/entity tuples
    entities = [(word,_,tag) for word, _, tag in iob_tags]
    return entities

# Applying NLTK's NER
df['Nltk_pos_ner'] = df['Sentence'].apply(nltk_ner)
df['Nltk_pos_ner'] = df['Nltk_pos_ner'].apply(lambda x : [(t[0], t[1], nltk_to_spacy.get(t[2], t[2])) for t in x])

df.head()

Sentence_ID                                               Word  \
0      Sentence: 1  ['Thousands', 'of', 'demonstrators', 'have', '...   
1     Sentence: 10  ['Iranian', 'officials', 'say', 'they', 'expec...   
2    Sentence: 100  ['Helicopter', 'gunships', 'Saturday', 'pounde...   
3   Sentence: 1000  ['They', 'left', 'after', 'a', 'tense', 'hour-...   
4  Sentence: 10000  ['U.N.', 'relief', 'coordinator', 'Jan', 'Egel...   

                                                 POS  \
0  ['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...   
1  ['JJ', 'NNS', 'VBP', 'PRP', 'VBP', 'TO', 'VB',...   
2  ['NN', 'NNS', 'NNP', 'VBD', 'JJ', 'NNS', 'IN',...   
3  ['PRP', 'VBD', 'IN', 'DT', 'NN', 'JJ', 'NN', '...   
4  ['NNP', 'NN', 'NN', 'NNP', 'NNP', 'VBD', 'NNP'...   

                                                 Tag  \
0  ['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '...   
1  ['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '...   
2  ['O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', '...   
3  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...   
4  ['B-geo', 'O', 'O', 'B-per', 'I-per', 'O', 'B-...   

                                            Sentence  \
0  Thousands of demonstrators have marched throug...   
1  Iranian officials say they expect to get acces...   
2  Helicopter gunships Saturday pounded militant ...   
3  They left after a tense hour-long standoff wit...   
4  U.N. relief coordinator Jan Egeland said Sunda...   

                                       Spacy_pos_ner  \
0  [(Thousands, NOUN, CARDINAL), (of, ADP, ), (de...   
1  [(Iranian, ADJ, NORP), (officials, NOUN, ), (s...   
2  [(Helicopter, NOUN, ), (gunships, NOUN, ), (Sa...   
3  [(They, PRON, ), (left, VERB, ), (after, ADP, ...   
4  [(U.N., PROPN, ORG), (relief, NOUN, ), (coordi...   

                                     Dataset_pos_ner  \
0  [(Thousands, NNS, ), (of, IN, ), (demonstrator...   
1  [(Iranian, JJ, GPE), (officials, NNS, ), (say,...   
2  [(Helicopter, NN, ), (gunships, NNS, ), (Satur...   
3  [(They, PRP, ), (left, VBD, ), (after, IN, ), ...   
4  [(U.N., NNP, GPE), (relief, NN, ), (coordinato...   

                                        Nltk_pos_ner  
0  [(Thousands, NNS, ), (of, IN, ), (demonstrator...  
1  [(Iranian, JJ, GPE), (officials, NNS, ), (say,...  
2  [(Helicopter, NNP, ), (gunships, NNS, ), (Satu...  
3  [(They, PRP, ), (left, VBD, ), (after, IN, ), ...  
4  [(U.N., NNP, B-ORGANIZATION), (relief, NN, ), ...

In [ ]:
import pandas as pd
import nltk
from nltk.tag import StanfordNERTagger
import os

nltk.download('punkt')
nltk.download('maxent_ne_chunker')
nltk.download('words')
# path to the Stanford NER directory and jar file
stanford_ner_dir = '/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/stanford-ner-4.2.0/stanford-ner-2020-11-17'
stanford_ner_jar = os.path.join(stanford_ner_dir, 'stanford-ner.jar')

# path to the Stanford NER model file
stanford_ner_model = os.path.join(stanford_ner_dir, 'classifiers', 'english.all.3class.distsim.crf.ser.gz')

# Initialization of Stanford NER tagger
stanford_ner_tagger = StanfordNERTagger(stanford_ner_model, stanford_ner_jar)

def stanford_ner_pos(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Perform named entity recognition using Stanford NER
    ner_tags = stanford_ner_tagger.tag(tokens)
    # Perform part-of-speech tagging using NLTK POS tagger
    pos_tags = nltk.pos_tag(tokens)
    # Combine the token, POS tag, and NER tag into a tuple for each token
    combined_tags = [(token, pos_tag, ner_tag) for ((token, ner_tag), (_, pos_tag)) in zip(ner_tags, pos_tags)]
    return combined_tags

# Applying Stanford NER and NLTK POS to each sentence in the DataFrame
df['Stanford_pos_ner'] = df['Sentence'].apply(stanford_ner_pos)
df['Stanford_pos_ner'] = df['Stanford_pos_ner'].apply(lambda x : [(t[0], t[1], nltk_to_spacy.get(t[2], t[2])) for t in x])

df.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


Sentence_ID                                               Word  \
0      Sentence: 1  ['Thousands', 'of', 'demonstrators', 'have', '...   
1     Sentence: 10  ['Iranian', 'officials', 'say', 'they', 'expec...   
2    Sentence: 100  ['Helicopter', 'gunships', 'Saturday', 'pounde...   
3   Sentence: 1000  ['They', 'left', 'after', 'a', 'tense', 'hour-...   
4  Sentence: 10000  ['U.N.', 'relief', 'coordinator', 'Jan', 'Egel...   

                                                 POS  \
0  ['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...   
1  ['JJ', 'NNS', 'VBP', 'PRP', 'VBP', 'TO', 'VB',...   
2  ['NN', 'NNS', 'NNP', 'VBD', 'JJ', 'NNS', 'IN',...   
3  ['PRP', 'VBD', 'IN', 'DT', 'NN', 'JJ', 'NN', '...   
4  ['NNP', 'NN', 'NN', 'NNP', 'NNP', 'VBD', 'NNP'...   

                                                 Tag  \
0  ['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '...   
1  ['B-gpe', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '...   
2  ['O', 'O', 'B-tim', 'O', 'O', 'O', 'O', 'O', '...   
3  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...   
4  ['B-geo', 'O', 'O', 'B-per', 'I-per', 'O', 'B-...   

                                            Sentence  \
0  Thousands of demonstrators have marched throug...   
1  Iranian officials say they expect to get acces...   
2  Helicopter gunships Saturday pounded militant ...   
3  They left after a tense hour-long standoff wit...   
4  U.N. relief coordinator Jan Egeland said Sunda...   

                                       Spacy_pos_ner  \
0  [(Thousands, NOUN, CARDINAL), (of, ADP, ), (de...   
1  [(Iranian, ADJ, NORP), (officials, NOUN, ), (s...   
2  [(Helicopter, NOUN, ), (gunships, NOUN, ), (Sa...   
3  [(They, PRON, ), (left, VERB, ), (after, ADP, ...   
4  [(U.N., PROPN, ORG), (relief, NOUN, ), (coordi...   

                                     Dataset_pos_ner  \
0  [(Thousands, NNS, ), (of, IN, ), (demonstrator...   
1  [(Iranian, JJ, GPE), (officials, NNS, ), (say,...   
2  [(Helicopter, NN, ), (gunships, NNS, ), (Satur...   
3  [(They, PRP, ), (left, VBD, ), (after, IN, ), ...   
4  [(U.N., NNP, GPE), (relief, NN, ), (coordinato...   

                                        Nltk_pos_ner  \
0  [(Thousands, NNS, ), (of, IN, ), (demonstrator...   
1  [(Iranian, JJ, GPE), (officials, NNS, ), (say,...   
2  [(Helicopter, NNP, ), (gunships, NNS, ), (Satu...   
3  [(They, PRP, ), (left, VBD, ), (after, IN, ), ...   
4  [(U.N., NNP, B-ORGANIZATION), (relief, NN, ), ...   

                                    Stanford_pos_ner  
0  [(Thousands, NNS, ), (of, IN, ), (demonstrator...  
1  [(Iranian, JJ, ), (officials, NNS, ), (say, VB...  
2  [(Helicopter, NNP, ), (gunships, NNS, ), (Satu...  
3  [(They, PRP, ), (left, VBD, ), (after, IN, ), ...  
4  [(U.N., NNP, ORGANIZATION), (relief, NN, ), (c...

# Task 2: Document Classification (using traditional ML techniques)
Dataset:
Choose any one text dataset from here. Create a random sample of 20k from the chosen set for the task.
Each review has a rating. The task is to develop a classifier using ML methods to predict the rating.
More details in subtasks.
IT IS MANDATORY TO SUBMIT A SINGLE .py file with rollno1_rollno2_rollno3_t2.py for this TASK 2. (failing which task 2 will be awarded 0)

Sub-tasks:

Define your train-val split. [Report the split chosen.]
Which set of terms best describes your corpus? How did you arrive at it?
Define a text preprocessing pipeline, i.e., stopword removal, lower casing, punctuation removal etc. [Report your text preprocessing pipeline in the report].
Design a multi-class classifier to predict the actual rating using a bag of words with different weight calculation mechanisms, for each of the following models -
Naive Bayes classifier.
Gaussian NB.
Multinomial NB.
Decision tree.
Criteria: entropy
Criteria: gini
Random Forest
No of trees = 20
No of trees = 50
No of trees = 100

Provide comparative performance for each weighing mechanism using accuracy, and confusion matrix (class-wise).
Treat the problem as a binary classification problem using classes 1 and 2 as NEGATIVE and 3, 4, and 5 as POSITIVE.
Compare the performance of the classifiers using accuracy, Precision, Recall, F1 score and confusion matrix.
Report hyperparameters (if any)
Analyze the erroneous results model-wise, and provide your insights on why you think it is happening.

TEST - 20-30 unseen test samples shall be provided and you have to quickly run the best model (from model in the sub-task 7) and share the results [for one part of the evaluation] [You shall be given one hour after the assignment deadline to run inference on these unseen test samples]

NOTE: Refer to literature for feature selection mechanisms for sentiment analysis using statistical machine learning.


# Data-Preprocessing

In [6]:
# library
!pip install imbalanced-learn nlpaug
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 68.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [7]:
import os

# library
import pandas as pd
import gzip
import json

def parse(path):
  g = gzip.open(path, 'rb')
  cnt = 0
  for l in g:
    if cnt > 2000:
        break
    cnt += 1
    yield json.loads(l)

def getdata(path):
  i = 0
  data = {}
  for d in parse(path):
    data[i] = d
    i += 1
  return pd.DataFrame.from_dict(data, orient='index')

# directory path
# dir_path = '/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/'

# # get all file names in the directory
# files = os.listdir(dir_path)

# # filter out the required files
# files = [file for file in files if file.endswith('.gz')]

# # initialize an empty dataframe
# final_data = pd.DataFrame()

# # loop over the files and concatenate the dataframes
# for file in files:
#   path = dir_path + file
#   data = getdata(path)
#   data = data[data.reviewText.isna()==False].reset_index(drop=True)
#   final_data = pd.concat([final_data, data])

# # reset the index of the final dataframe
# final_data = final_data.reset_index(drop=True)
# final_data.to_csv('/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/data.csv')

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/data.csv')

In [8]:
data = data.head(20000)
data.shape

(20000, 12)

In [9]:
# Import necessary libraries
import spacy
import string

# Load Spacy model
nlp = spacy.load('en_core_web_sm')

def preprocessing(sentence):
    # Create a Doc object
    doc = nlp(sentence)

    # Generate list of tokens
    tokens = [token.text for token in doc]

    # Remove stop words and punctuation, and perform lemmatization
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct]
    text = ' '.join(i for i in tokens)

    return text

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [10]:
# Import necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from joblib import dump, load
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=12)

# Preprocess your text data
data['reviewText'] = data['reviewText'].apply(preprocessing)

# Map the ratings to binary classes
data['sentiment'] = data['overall']#.apply(lambda x: 1 if x > 2 else 0)

# check data balance
print(data['sentiment'].value_counts())

# Split the data into features (X) and target (y)
X = data['reviewText']
y = data['sentiment']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Bag of Words representation
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_train_bow, y_train_bow = smote.fit_resample(X_train_bow, y_train)
X_val_bow = vectorizer.transform(X_val)

# check data balance
print(y_train_bow.value_counts())

# Create a Bag of Words representation with TF-IDF weighting
smote = SMOTE(random_state=21)
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)

import pickle

# Save the fitted tfidf vectorizer to disk
with open('/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/tfidf_vectorizer.pkl', 'wb') as handle:
    pickle.dump(vectorizer, handle)

# Later on, load the fitted tfidf vectorizer from disk
with open('/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/tfidf_vectorizer.pkl', 'rb') as handle:
    loaded_vectorizer = pickle.load(handle)

# Use the loaded vectorizer to transform your test dataset
X_val_tfidf = loaded_vectorizer.transform(X_val)

X_train_tfidf, y_train_tfidf = smote.fit_resample(X_train_tfidf, y_train)

# check data balance
print(y_train_tfidf.value_counts())

sentiment
5    13775
4     3105
3     1466
1      949
2      705
Name: count, dtype: int64
sentiment
5    11026
4    11026
2    11026
3    11026
1    11026
Name: count, dtype: int64
sentiment
5    11026
4    11026
2    11026
3    11026
1    11026
Name: count, dtype: int64


# Naive Bayes classifier

## Training

In [ ]:
# Train the Gaussian Naive Bayes model
gnb = GaussianNB()
gnb.fit(X_train_bow.toarray(), y_train_bow)

# Train the Gaussian Naive Bayes model
gnb_tfidf = GaussianNB()
gnb_tfidf.fit(X_train_tfidf.toarray(), y_train_tfidf)

# Train the Multinomial Naive Bayes model
mnb = MultinomialNB()
mnb.fit(X_train_bow, y_train_bow)

# Train the Multinomial Naive Bayes model
mnb_tfidf = MultinomialNB()
mnb_tfidf.fit(X_train_tfidf, y_train_tfidf)

# Trained Model pickle
dump(gnb, '/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/gnb.joblib')
dump(gnb_tfidf, '/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/gnb_tfidf.joblib')

dump(mnb, '/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/mnb.joblib')
dump(mnb_tfidf, '/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/mnb_tfidf.joblib')

['/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/mnb_tfidf.joblib']

## Inference

In [ ]:
# load trained models
gnb = load('/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/gnb.joblib')
gnb_tfidf = load('/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/gnb_tfidf.joblib')

mnb = load('/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/mnb.joblib')
mnb_tfidf = load('/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/mnb_tfidf.joblib')

# Make predictions on the validation set
y_pred_gnb = gnb.predict(X_val_bow.toarray())
y_pred_mnb = mnb.predict(X_val_bow)

# Make predictions on the validation set
y_pred_gnb_tfidf = gnb_tfidf.predict(X_val_tfidf.toarray())
y_pred_mnb_tfidf = mnb_tfidf.predict(X_val_tfidf)

## Evaluation

In [ ]:
# Calculate metrics for the Gaussian Naive Bayes model
accuracy_gnb = accuracy_score(y_val, y_pred_gnb)
precision_gnb = precision_score(y_val, y_pred_gnb, pos_label=1)
recall_gnb = recall_score(y_val, y_pred_gnb, pos_label=1)
f1_gnb = f1_score(y_val, y_pred_gnb, pos_label=1)
confusion_gnb = confusion_matrix(y_val, y_pred_gnb)

# Calculate metrics for the Gaussian Naive Bayes model
accuracy_gnb_tfidf = accuracy_score(y_val, y_pred_gnb_tfidf)
precision_gnb_tfidf = precision_score(y_val, y_pred_gnb_tfidf, pos_label=1)
recall_gnb_tfidf = recall_score(y_val, y_pred_gnb_tfidf, pos_label=1)
f1_gnb_tfidf = f1_score(y_val, y_pred_gnb_tfidf, pos_label=1)
confusion_gnb_tfidf = confusion_matrix(y_val, y_pred_gnb_tfidf)

# Calculate metrics for the Multinomial Naive Bayes model
accuracy_mnb = accuracy_score(y_val, y_pred_mnb)
precision_mnb = precision_score(y_val, y_pred_mnb, pos_label=1)
recall_mnb = recall_score(y_val, y_pred_mnb, pos_label=1)
f1_mnb = f1_score(y_val, y_pred_mnb, pos_label=1)
confusion_mnb = confusion_matrix(y_val, y_pred_mnb)

# Calculate metrics for the Multinomial Naive Bayes model
accuracy_mnb_tfidf = accuracy_score(y_val, y_pred_mnb_tfidf)
precision_mnb_tfidf = precision_score(y_val, y_pred_mnb_tfidf, pos_label=1)
recall_mnb_tfidf = recall_score(y_val, y_pred_mnb_tfidf, pos_label=1)
f1_mnb_tfidf = f1_score(y_val, y_pred_mnb_tfidf, pos_label=1)
confusion_mnb_tfidf = confusion_matrix(y_val, y_pred_mnb_tfidf)

# Print the metrics
print("Gaussian NB: Accuracy = {}, Precision = {}, Recall = {}, F1 Score = {}, Confusion Matrix = {}"
      .format(accuracy_gnb, precision_gnb, recall_gnb, f1_gnb, confusion_gnb))
print("Multinomial NB: Accuracy = {}, Precision = {}, Recall = {}, F1 Score = {}, Confusion Matrix = {}"
      .format(accuracy_mnb, precision_mnb, recall_mnb, f1_mnb, confusion_mnb))

# Print the metrics
print("Gaussian NB: Accuracy = {}, Precision = {}, Recall = {}, F1 Score = {}, Confusion Matrix = {}"
      .format(accuracy_gnb_tfidf, precision_gnb_tfidf, recall_gnb_tfidf, f1_gnb, confusion_gnb_tfidf))
print("Multinomial NB: Accuracy = {}, Precision = {}, Recall = {}, F1 Score = {}, Confusion Matrix = {}"
      .format(accuracy_mnb_tfidf, precision_mnb_tfidf, recall_mnb_tfidf, f1_mnb_tfidf, confusion_mnb_tfidf))

Gaussian NB: Accuracy = 0.50125, Precision = 0.9140859140859141, Recall = 0.5009581166164796, F1 Score = 0.6472148541114059, Confusion Matrix = [[ 175  172]
 [1823 1830]]
Multinomial NB: Accuracy = 0.87225, Precision = 0.950401376146789, Recall = 0.9074733096085409, F1 Score = 0.9284413947626383, Confusion Matrix = [[ 174  173]
 [ 338 3315]]
Gaussian NB: Accuracy = 0.51975, Precision = 0.9163461538461538, Recall = 0.5217629345743224, F1 Score = 0.6472148541114059, Confusion Matrix = [[ 173  174]
 [1747 1906]]
Multinomial NB: Accuracy = 0.85375, Precision = 0.9705521472392638, Recall = 0.8661374212975637, F1 Score = 0.9153768262693476, Confusion Matrix = [[ 251   96]
 [ 489 3164]]


# Decision tree.

## Training

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Train the model
clf_entropy = DecisionTreeClassifier(criterion='entropy', random_state=42)
clf_entropy.fit(X_train_tfidf, y_train_tfidf)

# Train the model
clf_gini = DecisionTreeClassifier(criterion='gini', random_state=42)
clf_gini.fit(X_train_tfidf, y_train_tfidf)

# Trained Model pickle
dump(clf_entropy, '/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/clf_entropy.joblib')
dump(clf_gini, '/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/clf_gini.joblib')

['/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/clf_gini.joblib']

## Inference

In [ ]:
# load trained models
clf_entropy = load('/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/clf_entropy.joblib')
clf_gini = load('/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/clf_gini.joblib')

# Predict the labels on the test set
y_pred_entropy = clf_entropy.predict(X_val_tfidf)
# Predict the labels on the test set
y_pred_gini = clf_gini.predict(X_val_tfidf)


# Print the classification report
print('clf_entropy-',classification_report(y_val, y_pred_entropy))
print('clf_gini-',classification_report(y_val, y_pred_gini))

clf_entropy-               precision    recall  f1-score   support

           0       0.34      0.43      0.38       347
           1       0.94      0.92      0.93      3653

    accuracy                           0.88      4000
   macro avg       0.64      0.68      0.66      4000
weighted avg       0.89      0.88      0.88      4000

clf_gini-               precision    recall  f1-score   support

           0       0.30      0.41      0.35       347
           1       0.94      0.91      0.93      3653

    accuracy                           0.87      4000
   macro avg       0.62      0.66      0.64      4000
weighted avg       0.89      0.87      0.88      4000



## Evaluation

In [ ]:
# Calculate metrics for the Gaussian Naive Bayes model
accuracy_entropy = accuracy_score(y_val, y_pred_entropy)
precision_entropy = precision_score(y_val, y_pred_entropy, pos_label=1)
recall_entropy = recall_score(y_val, y_pred_entropy, pos_label=1)
f1_entropy = f1_score(y_val, y_pred_entropy, pos_label=1)
confusion_entropy = confusion_matrix(y_val, y_pred_entropy)

# Calculate metrics for the Gaussian Naive Bayes model
accuracy_gini = accuracy_score(y_val, y_pred_gini)
precision_gini = precision_score(y_val, y_pred_gini, pos_label=1)
recall_gini = recall_score(y_val, y_pred_gini, pos_label=1)
f1_gini = f1_score(y_val, y_pred_gini, pos_label=1)
confusion_gini = confusion_matrix(y_val, y_pred_gini)

# Print the metrics
print("Decision Tree Entropy: Accuracy = {}, Precision = {}, Recall = {}, F1 Score = {}, Confusion Matrix = {}"
      .format(accuracy_entropy, precision_entropy, recall_entropy, f1_entropy, confusion_entropy))
print("Decision Tree Gini: Accuracy = {}, Precision = {}, Recall = {}, F1 Score = {}, Confusion Matrix = {}"
      .format(accuracy_gini, precision_gini, recall_gini, f1_gini, confusion_gini))

Decision Tree Entropy: Accuracy = 0.87825, Precision = 0.9446629213483146, Recall = 0.9206131946345469, F1 Score = 0.9324830167752668, Confusion Matrix = [[ 150  197]
 [ 290 3363]]
Decision Tree Gini: Accuracy = 0.866, Precision = 0.9421276595744681, Recall = 0.9091157952367916, F1 Score = 0.925327389244915, Confusion Matrix = [[ 143  204]
 [ 332 3321]]


# Random Forest

## Training

In [11]:
from sklearn.ensemble import RandomForestClassifier

# List of number of trees
n_trees = [20, 50, 100]
rf_models = {}

for n in n_trees:
  # Initialize the model
  clf_rf = RandomForestClassifier(n_estimators=n, random_state=42)
  # Train the model
  clf_rf.fit(X_train_tfidf, y_train_tfidf)

  # Save the model using joblib
  dump(clf_rf, f'/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/clf_rf_{n}.joblib')

## Inference

In [13]:
# load trained models
model_inference = {}
for n in n_trees:
  clf_rf = load(f'/content/drive/MyDrive/Colab Notebooks/NLU_assignment/Assignment-1/clf_rf_{n}.joblib')
  # Perform inference
  y_pred_rf = clf_rf.predict(X_val_tfidf)
  model_inference[f'y_pred_rf_{n}'] = y_pred_rf
  # Evaluate the model
  print(f"Classification Report for {n} trees:")
  # print(classification_report(y_val, y_pred_rf))

Classification Report for 20 trees:
Classification Report for 50 trees:
Classification Report for 100 trees:


## Evaluation

In [14]:
# Calculate metrics for the Gaussian Naive Bayes model

y_pred_rf = model_inference['y_pred_rf_20']
accuracy_rf_20 = accuracy_score(y_val, y_pred_rf)
precision_rf_20 = precision_score(y_val, y_pred_rf, pos_label=1)
recall_rf_20 = recall_score(y_val, y_pred_rf, pos_label=1)
f1_rf_20 = f1_score(y_val, y_pred_rf, pos_label=1)
confusion_rf_20 = confusion_matrix(y_val, y_pred_rf)

# Calculate metrics for the Gaussian Naive Bayes model
y_pred_rf = model_inference['y_pred_rf_50']
accuracy_rf_50 = accuracy_score(y_val, y_pred_rf)
precision_rf_50 = precision_score(y_val, y_pred_rf, pos_label=1)
recall_rf_50 = recall_score(y_val, y_pred_rf, pos_label=1)
f1_rf_50 = f1_score(y_val, y_pred_rf, pos_label=1)
confusion_rf_50 = confusion_matrix(y_val, y_pred_rf)

y_pred_rf = model_inference['y_pred_rf_100']
accuracy_rf_100 = accuracy_score(y_val, y_pred_rf)
precision_rf_100 = precision_score(y_val, y_pred_rf, pos_label=1)
recall_rf_100 = recall_score(y_val, y_pred_rf, pos_label=1)
f1_rf_100 = f1_score(y_val, y_pred_rf, pos_label=1)
confusion_rf_100 = confusion_matrix(y_val, y_pred_rf)

# Print the metrics
print("Random Forest no_trees-20: Accuracy = {}, Precision = {}, Recall = {}, F1 Score = {}, Confusion Matrix = {}"
      .format(accuracy_rf_20, precision_rf_20, recall_rf_20, f1_rf_20, confusion_rf_20))
print("Random Forest no_trees-50: Accuracy = {}, Precision = {}, Recall = {}, F1 Score = {}, Confusion Matrix = {}"
      .format(accuracy_rf_50, precision_rf_50, recall_rf_50, f1_rf_50, confusion_rf_50))
print("Random Forest no_trees-100: Accuracy = {}, Precision = {}, Recall = {}, F1 Score = {}, Confusion Matrix = {}"
      .format(accuracy_rf_100, precision_rf_100, recall_rf_100, f1_rf_100, confusion_rf_100))

ValueError: Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted'].

# Summary

In [ ]:
# Define your models and their metrics
model_metrics = [
    f"Gaussian NB-CountVectorizer-{accuracy_gnb}-{precision_gnb}-{recall_gnb}-{f1_gnb}-{confusion_gnb}",
    f"Gaussian NB-TfidfVectorizer-{accuracy_gnb_tfidf}-{precision_gnb_tfidf}-{recall_gnb_tfidf}-{f1_gnb_tfidf}-{confusion_gnb_tfidf}",
    f"Multinomial NB-CountVectorizer-{accuracy_mnb}-{precision_mnb}-{recall_mnb}-{f1_mnb}-{confusion_mnb}",
    f"Multinomial NB-TfidfVectorizer-{accuracy_mnb_tfidf}-{precision_mnb_tfidf}-{recall_mnb_tfidf}-{f1_mnb_tfidf}-{confusion_mnb_tfidf}",
    f"Decision Tree Entropy-TfidfVectorizer-{accuracy_entropy}-{precision_entropy}-{recall_entropy}-{f1_entropy}-{confusion_entropy}",
    f"Decision Tree Gini-TfidfVectorizer-{accuracy_gini}-{precision_gini}-{recall_gini}-{f1_gini}-{confusion_gini}",
    f"Random Forest no_trees20-TfidfVectorizer-{accuracy_rf_20}-{precision_rf_20}-{recall_rf_20}-{f1_rf_20}-{confusion_rf_20}",
    f"Random Forest no_trees50-TfidfVectorizer-{accuracy_rf_50}-{precision_rf_50}-{recall_rf_50}-{f1_rf_50}-{confusion_rf_50}",
    f"Random Forest no_trees100-TfidfVectorizer-{accuracy_rf_100}-{precision_rf_100}-{recall_rf_100}-{f1_rf_100}-{confusion_rf_100}"
]

# Split the metrics and create a DataFrame
summary = []
for metrics in model_metrics:
    summary.append(metrics.split('-'))

summary = pd.DataFrame(summary, columns=['Model', 'BOW_Wt', 'Accuracy', 'Precision', 'Recall', 'F1Score', 'Confusion Matrix'])

summary

Model           BOW_Wt Accuracy           Precision  \
0                Gaussian NB  CountVectorizer  0.50125  0.9140859140859141   
1                Gaussian NB  TfidfVectorizer  0.51975  0.9163461538461538   
2             Multinomial NB  CountVectorizer  0.87225   0.950401376146789   
3             Multinomial NB  TfidfVectorizer  0.85375  0.9705521472392638   
4      Decision Tree Entropy  TfidfVectorizer  0.87825  0.9446629213483146   
5         Decision Tree Gini  TfidfVectorizer    0.866  0.9421276595744681   
6   Random Forest no_trees20  TfidfVectorizer   0.8975  0.9424283765347885   
7   Random Forest no_trees50  TfidfVectorizer    0.909  0.9419510884170922   
8  Random Forest no_trees100  TfidfVectorizer    0.913  0.9417268110130981   

               Recall             F1Score             Confusion Matrix  
0  0.5009581166164796  0.6472148541114059  [[ 175  172]\n [1823 1830]]  
1  0.5217629345743224  0.6649223792080935  [[ 173  174]\n [1747 1906]]  
2  0.9074733096085409  0.9284413947626383  [[ 174  173]\n [ 338 3315]]  
3  0.8661374212975637  0.9153768262693476  [[ 251   96]\n [ 489 3164]]  
4  0.9206131946345469  0.9324830167752668  [[ 150  197]\n [ 290 3363]]  
5  0.9091157952367916   0.925327389244915  [[ 143  204]\n [ 332 3321]]  
6  0.9455242266630167  0.9439737633233123  [[ 136  211]\n [ 199 3454]]  
7  0.9594853545031481  0.9506373745592622  [[ 131  216]\n [ 148 3505]]  
8  0.9644128113879004  0.9529348120097376  [[ 129  218]\n [ 130 3523]]